<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8%EB%A1%9C_%EC%98%81%ED%99%94_%EC%9E%A5%EB%A5%B4_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
mvcp = mv.copy()

mvcp[mv.columns[mv.columns.str.contains('_비율')]] = mv_clf
mvcp['장르'] = mv['장르'].str.split(',').str[0]


def check_genre(x) :
  if x in mvcp['장르'].value_counts()[mvcp['장르'].value_counts() < 4].index :
    return 'other'
  else :
    return x

mvcp['장르'] = mvcp['장르'].map(check_genre)

#mvcp['배급'].value_counts()[mvcp['배급'].value_counts() >= 30]





def check_distributor(x) :
  if x in mvcp['배급'].value_counts()[mvcp['배급'].value_counts() < 30].index :
    return 'other'
  else :
    return x

mvcp['배급'] = mvcp['배급'].map(check_distributor)



mvcp['개봉'] = mvcp['개봉'].str[5:7].astype('int8')
def check_season(x) :
  if x > 2 :
    return '봄'
  elif x > 5 : 
    return '여름'
  elif x > 8 :
    return '가을'
  else : 
    return '겨울'

mvcp['개봉시즌'] = mvcp['개봉'].map(check_season)

In [ ]:
# 감독과 평점의 관계에서는 감독이 평점과 관련이 있다는 것을 보여주기 위해 타겟 인코딩
# 감독은 평점과 장르 예측 모두에서 유용하게 쓰일 수 있으나, 현재 데이터의 상황은 그리 좋아보이지 않음.(샘플이 적음)
## 인코딩으로도 한계가 잇는 경우 더 많은 데이터를 확보하거나 해당 특성을 제외하도록 한다.

mvcp2 = mvcp[['개봉시즌', '장르', '등급', '배급', 'OST', '영상미', '스토리', '연기', '연출', '남자평균', '여자평균', '10대_비율', '20대_비율', '30대_비율', '40대_비율', '50대_비율']]

mvcp3 = mvcp2[mvcp2[mvcp2.columns[mvcp2.columns.str.contains('_비율')]].sum(axis=1) > 0]

mvcp4 = mvcp3.dropna()

mvcp4_cate = mvcp4.select_dtypes('object').drop('장르', axis=1)
mvcp4_num = mvcp4.select_dtypes(include=['float64'])
mvcp4_target = mvcp4['장르'].dropna()

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

full_pipeline = ColumnTransformer([
    ("num", StandardScaler(), mvcp4_num.columns.values),
    ("cat", OneHotEncoder(), mvcp4_cate.columns.values)
])


In [ ]:
from sklearn.model_selection import train_test_split

mvcp4_rst = full_pipeline.fit_transform(mvcp4)
trd, tsd, trt, tst = train_test_split(mvcp4_rst, mvcp4_target, train_size=0.8, stratify=mvcp4_target, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=10000)

lr.fit(trd, trt)


LogisticRegression(max_iter=10000)

In [ ]:
lr.score(trd, trt), lr.score(tsd, tst),

(0.5932914046121593, 0.4666666666666667)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=8, random_state=42, max_features=0.5, n_estimators=1000)

rfc.fit(trd, trt)
rfc.score(trd, trt), rfc.score(tsd, tst)

(0.9412997903563941, 0.5416666666666666)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [100, 200, 400, 800],
    'max_depth' : [5, 10, 15, 20],
    'max_features' : ['auto', 0.1, 0.5, 0.8, 1]
}

gs = GridSearchCV(rfc, param_grid=param_grid, cv=5)

In [ ]:
gs.fit(trd, trt)

In [ ]:
gs.best_params_
best_rfc = gs.best_estimator_

best_rfc.score(trd, trt), best_rfc.score(tsd, tst)

(0.9811320754716981, 0.475)

In [ ]:
from sklearn.impute import SimpleImputer

mvcp2 = mvcp[['개봉시즌', '장르', '등급', '배급', 'OST', '영상미', '스토리', '연기', '연출', '남자평균', '여자평균', '10대_평점', '20대_평점',
       '30대_평점', '40대_평점', '50대_평점', '10대_비율', '20대_비율', '30대_비율', '40대_비율', '50대_비율']]

mvcp2_cate = mvcp2.select_dtypes('object').drop('장르', axis=1)
mvcp2_num = mvcp2.select_dtypes(include=['float64'])
mvcp2_target = mvcp2['장르']

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

cate_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])


In [ ]:

full_pipeline2 = ColumnTransformer([
    ("num", num_pipeline, mvcp2_num.columns.values),
    ("cat", cate_pipeline, mvcp2_cate.columns.values)
])

In [ ]:
mvcp2_rst = full_pipeline2.fit_transform(mvcp2)
simputer = SimpleImputer(strategy='most_frequent')
mvcp2_target2 = simputer.fit_transform(mvcp2_target.values.reshape(len(mvcp2_target), -1))

trd2, tsd2, trt2, tst2 = train_test_split(mvcp2_rst, mvcp2_target2.reshape(len(mvcp2_target2)), train_size=0.8, stratify=mvcp2_target2.reshape(len(mvcp2_target2)), random_state=42)


In [ ]:
np.unique(mvcp2_target2.reshape(len(mvcp2_target2)))

array(['SF', 'other', '가족', '공연실황', '공포', '다큐멘터리', '드라마', '멜로/로맨스', '모험',
       '뮤지컬', '미스터리', '범죄', '스릴러', '애니메이션', '액션', '전쟁', '코미디', '판타지'],
      dtype=object)

In [ ]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=10, max_features='sqrt')

rfc.fit(trd2, trt2)
rfc.score(trd2, trt2), rfc.score(tsd2, tst2)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rfc, trd2, trt2, scoring='accuracy', cv=3)

In [ ]:
scores

array([0.47003155, 0.45425868, 0.46518987])

In [ ]:
rfc2 = RandomForestClassifier(class_weight='balanced')
scores = cross_val_score(rfc2, trd2, trt2, scoring='accuracy', cv=5)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
scores

array([0.43157895, 0.44736842, 0.44736842, 0.46315789, 0.47368421])

In [ ]:
rfc2.fit(trd2, trt2)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(rfc2.predict(tsd2[:10]), tst2[:10]))
